<a href="https://colab.research.google.com/github/A1exanderBates/CryptopunkMachineLearningTF/blob/main/CryptoCNNwithTF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cryptopunk Convolutional Neural Network with Tensorflow

You might have heard of the recent NFT (non-fungible token) craze. Cryptopunks are example NFT assets that one can buy. As of this writing, the cheapest Cryptopunk is worth over $40,000 dollars. The punks each have attributes, like a mustache or hairstyle. I will be training a conv net to classify punks by attributes. First I will download all of the cryptopunks, which are in a giant single image file. Then I will separate them into the individual punks. Next I  will download a database of attributes. You will then build a tensorflow model to predict the attributes given the images.

In [ ]:
import urllib.request
import PIL
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Import the image of all of the cryptopunks.

In [ ]:
imgURL = "https://raw.githubusercontent.com/larvalabs/cryptopunks/master/punks.png"
urllib.request.urlretrieve(imgURL, "cryptoPunksAll.jpg")

img = PIL.Image.open("cryptoPunksAll.jpg").convert("RGB")
img

It looks like there's 100x100=10,000 crypto punks each one being a 24x24 (x3 color channels) image.

In [ ]:
img.size

Convert to a numpy array and visualize some. Here's punk 0.

In [ ]:
imgArray = np.asarray(img)
plt.imshow(imgArray[0 : 23, 0 : 23, ])
plt.xticks([])
plt.yticks([])

In [ ]:
#Plot out the first 25 punks
plt.figure(figsize=(10,10))
for i in range(25): 
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  a, b = (24 * i), (24 * (i + 1))
  plt.imshow(imgArray[0 : 23, a : b, :])

In [ ]:
# Reordering the array
finalArray = np.empty((24,24, 3, 10000))
for i in range(100):
  for j in range(100):
    a, b = 24 * i, 24 * (i + 1)  
    c, d = 24 * j, 24 * (j + 1) 
    idx = j + i * (100)
    finalArray[:,:,:,idx] = imgArray[a:b,c:d,:]

## double check with above
## apparently the .astype('uint8') is needed to display correctly now
plt.figure(figsize=(10,10))
for i in range(25): 
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(finalArray[:,:,:,i].astype('uint8'))

In [ ]:
finalArray.shape

In [ ]:
# Reading in CSV files
baseUrl = "https://raw.githubusercontent.com/cryptopunksnotdead/punks/master/"
for i in range(0,10000, 1000):
  url = baseUrl+str(i)+"-"+str(i + 999)+".csv"
  print(url)
  if (i == 0):
    dat = pd.read_csv(url)
  else :
    dat = pd.concat ([dat, pd.read_csv(url)], 
                     join = 'inner',
                     ignore_index = True)

In [ ]:
dat.head()

In [ ]:
# Converting array into string for analysis of accessories
dat = dat.assign(earring= (dat[' accessories'].str.find("Earring") > 0) * 1)
dat.earring.value_counts()

In [ ]:
dat.shape

# The Convolutional Network

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

## Prepping the Data

In [ ]:
## Splitting the Data
train_images = finalArray[:, :, :, 0:7000]
train_labels = dat.earring[0:7000]
test_images = finalArray[:, :, :, 7000:]
test_labels = dat.earring[7000:]

## Changing dims for model fit
train_images = np.transpose(train_images, (3, 0, 1, 2))
test_images = np.transpose(test_images, (3, 0, 1, 2))

In [ ]:
#train_images.shape
#test_images.shape
#train_labels.shape
test_labels.shape

## Building the Model

### Creating the conv base

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(24, 24, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

### Adding dense layers on top

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

# Summary
model.summary()

### Compiling and training the model

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=5, 
                    validation_data=(test_images, test_labels))

### Evaluating the model

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

**Test Accuracy**

In [ ]:
print(test_acc*100)